In [1]:
!pip install git+https://github.com/d2l-ai/d2l-en # installing d2l
!pip install -U mxnet-cu101mkl==1.6.0  # updating mxnet to at least v1.6

  Cloning https://github.com/d2l-ai/d2l-en to /tmp/pip-req-build-fx28ksdj
  Running command git clone -q https://github.com/d2l-ai/d2l-en /tmp/pip-req-build-fx28ksdj
  Created wheel for d2l: filename=d2l-0.13.2-cp36-none-any.whl size=48945 sha256=045893f05f27b4ffb8969d480555606f9921ae9d4edaf6d564b35b395bdf5357
  Stored in directory: /tmp/pip-ephem-wheel-cache-av4yg3zp/wheels/82/6f/b8/8422c7ed664272685e3d139b127f89df39d934704a14b2156c
Successfully built d2l
     |████████████████████████████████| 711.0MB 23kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
import pandas as pd
from d2l import mxnet as d2l
from mxnet import gluon
import numpy as np
import gzip
from collections import defaultdict
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt

In [3]:
# Import users and items

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
data = []
for l in readGz("/content/train.json.gz"):
    data.append(l)  
users = []
for l in readGz("train.json.gz"):
    users.append(l['reviewerID'])
    
items = []
for l in readGz("train.json.gz"):
    items.append(l['itemID'])
ratings = []
for l in readGz("train.json.gz"):
    ratings.append(l['rating'])

In [4]:
def split_data_ml100k(data, num_users, num_items,
                      split_mode='random', test_ratio=0.2):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating = line[1], line[2], line[3]
            train_items.setdefault(u, []).append((u, i, rating))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [True if x == 1 else False for x in np.random.uniform(
            0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data

In [5]:
data=pd.DataFrame(data)

In [6]:
data=data.drop(columns=['reviewTime','reviewText','helpful','reviewHash',	'categories',	'unixReviewTime','helpful','summary',	'categoryID',	'price'])

In [7]:
users_df = pd.DataFrame(users)
items_df = pd.DataFrame(items)
ratings_df = pd.DataFrame(ratings)

In [8]:
num_users = users_df[0].unique().shape[0]
num_items = items_df[0].unique().shape[0]

In [9]:
train_data, test_data = split_data_ml100k(data, num_users, num_items,
    test_ratio=0.2)

In [10]:
alpha=train_data['rating'].mean()
alpha

4.233845537814705

In [11]:
user = {}
item = {}
user_index = 0
item_index = 0


for l in readGz("train.json.gz"):
    
    if l['reviewerID'] not in user:
        user[l['reviewerID']] = user_index; 
        user_index+=1
    
    if l['itemID'] not in item:
        item[l['itemID']] = item_index; 
        item_index+=1
        

## test file
file = open('pairs_Rating.txt', 'r')

for l in file.readlines():
    if l.startswith("reviewerID"):
        continue

    u,i = l.strip().split('-')
    if u not in user:
        user[u] = user_index;
        user_index+=1
    
    if i not in item:
        item[i] = item_index; 
        item_index+=1
file.close()

In [12]:
train_user_diction = {};   ## dict of items per user
train_item_diction = {};   ## dict of users per item
train_ratings = [];    ## labels for train data
user_max=0;     
item_max=0;     


In [13]:
for l in train_data.itertuples():
	user_id = int(user[l[1]])
	item_id = int(item[l[2]])
	rating = float(l[3])	
 
	train_ratings.append(rating)
	
	if(user_max<user_id):
		user_max = user_id;
			
	if(item_max<item_id):
		item_max = item_id;
			
	if user_id not in train_user_diction:
		train_user_diction[user_id] = 1;
	else:
		train_user_diction[user_id]+=1;
			
	if item_id not in train_item_diction:
		train_item_diction[item_id] = 1;
	else:
		train_item_diction[item_id]+=1;
		




In [14]:
## test data for rating prediction

for l in open('pairs_Rating.txt', 'r'):
	if l.startswith("reviewerID"):
			continue
	u,i = l.strip().split('-')
	user_id = int(user[u])
	item_id = int(item[i])

	
	if(user_max<user_id):
		user_max = user_id;
			
	if(item_max<item_id):
		item_max = item_id;

In [15]:
num_users = user_max+1;
num_items = item_max+1;
print('number of users: %i, number of items: %i' % (num_users, num_items))

number of users: 39283, number of items: 19915


In [16]:
test_ratings = []  ## labels for test data
for l in test_data.itertuples():
  rating = float(l[3])
  test_ratings.append(rating)

In [17]:
#Initializing Parameters for training
lmbda = 1 # Regularisation weight
k = 40  # Latent space dimension
epochs = 20  # Epochs
lr = 0.001  # Learning rate
mf = 0.01   # multiplication factor

In [18]:
# Create training interaction matrix
import random
R = np.zeros((num_users, num_items))
m, n = R.shape  
for l in train_data.itertuples():
  user_id = int(user[l[1]])
  item_id = int(item[l[2]])
  rating = float(l[3])
  R[user_id,item_id] = rating


beta_u = mf*np.zeros((num_users,1))    ## user bias
beta_i = mf*np.zeros((num_items,1))    ## item bias

P =  mf*np.random.randn(k,m) # Latent user matrix
Q =  mf*np.random.randn(k,n) # Latent item matrix

beta_u = beta_u.flatten()
beta_i = beta_i.flatten()

#Only consider non-zero matrix 
users,items = R.nonzero()      

In [25]:
## Training Loop to optimize functions
for epoch in range(epochs):
    for u, i in zip(users,items):
        e = R[u, i] - (alpha + beta_u[u] + beta_i[i] + np.dot(P[:,u].T,Q[:,i]))  # gradient error
        beta_i[i] += lr*(e - lmbda*beta_i[i])
        beta_u[u] += lr*(e - lmbda*beta_u[u])
        P[:,u] += lr * ( e * Q[:,i] - lmbda * P[:,u]) # Update user matrix
        Q[:,i] += lr * ( e * P[:,u] - lmbda * Q[:,i])  # Update item matrix
    
    index = 0
    diff = 0

    for l in train_data.itertuples():
      u = int(user[l[1]])
      i = int(item[l[2]])
      r = float(l[3])
      rating = min(40,(alpha+ beta_u[u] + beta_i[i] + np.dot(P[:,u].T,Q[:,i])))
      diff += (train_ratings[index] - rating)**2
      index+=1

    mse = diff/len(train_ratings)
    print(epoch, mse)
    
    


predict = []    ## predicted labels    

0 0.9670428194633509
1 0.9648947857653636
2 0.9627794905553078
3 0.9606962159892233
4 0.9586442663422042
5 0.956622967084366
6 0.9546316640070512
7 0.9526697223957652
8 0.9507365262467499
9 0.948831477524144
10 0.9469539954555639
11 0.945103515863171
12 0.9432794905282562
13 0.941481386587227
14 0.9397086859569709
15 0.9379608847879906
16 0.9362374929435586
17 0.934538033503259
18 0.9328620422897346
19 0.9312090674170168


In [26]:
#Predicting test data ratings

    
index = 0
diff = 0
for l in test_data.itertuples():
      u = int(user[l[1]])
      i = int(item[l[2]])
      r = float(l[3])
      rating = min(40,(alpha+ beta_u[u] + beta_i[i] + np.dot(P[:,u].T,Q[:,i])))
      diff += (test_ratings[index] - rating)**2
      index+=1

mse = diff/len(test_ratings)
print(epoch, mse)
  

19 1.1209336434766692


In [27]:
predict = []    ## predicted ratings
user_index=0
item_index=0
for l in open('pairs_Rating.txt', 'r'):
    if l.startswith("reviewerID"):
        continue
    u,i = l.strip().split('-')
    if u not in user:
        user[u] = user_index; user_index+=1
    
    if i not in item:
        item[i] = item_index; item_index+=1
    ut=user[u]
    it=item[i]
    predict.append(min(10,(alpha+ beta_u[ut] + beta_i[it] + np.dot(P[:,ut].T,Q[:,it]))))
    
file.close()

In [28]:
predictb = open("predictions_Rating.txt", 'w')
index=0
for l in open("pairs_Rating.txt"):  
  if l.startswith("reviewerID"):    
    #header    
    predictb.write(l)    
    continue  
  u,i = l.strip().split('-')     
  predictb.write(u + '-' + i + ',' + str(predict[index]) + '\n')  
  index+=1
predictb.close()

In [ ]:
for epoch in range(epochs):
  for u, i in zip(users,items):

      e = R[u, i] - (alpha + beta_u[u] + beta_i[i] + np.dot(P[:,u].T,Q[:,i]))  # Calculate error for gradient
      beta_i[i] += lr*(e - lmbda*beta_i[i])
      beta_u[u] += lr*(e - lmbda*beta_u[u])
      P[:,u] += lr * ( e * Q[:,i] - lmbda * P[:,u]) # Update user matrix
      Q[:,i] += lr * ( e * P[:,u] - lmbda * Q[:,i])  # Update item matrix